# Actividad 3: Retreival-Augmented Generation (RAG)

En esta actividad vas a descubrir cómo se implementa un sistema de RAG. 
La evaluación de esta actividad considera que puedas producir respuestas para un contexto en particular, tomando en cuenta distintas sensibilidades a la hora de entregar informaicón al LLM. 

### Imports

Puedes usar el yaml proporcionado en el repositorio para instalar un ambiente que tenga todo lo necesario para estas importaciones. 

In [1]:
import os
from sentence_transformers import SentenceTransformer
from langchain_chroma import Chroma
from langchain_openai import ChatOpenAI
from langchain_core.embeddings import Embeddings

from transformers import AutoTokenizer
from langchain.document_loaders import TextLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_text_splitters import CharacterTextSplitter
from langchain_community.document_loaders import TextLoader
from langchain.schema import Document

In [2]:
### Poner la llave aquí! 


## El modelo que computa los vectores 

SentenceTransformer tiene los modelos necesarios para realizar el computo de string a vectores. 
Pero Langchain espera que le pasemos un modelo con métodos embed_documents() y embed_query(). 
Como vemos más abajo, en el modelo que usamos (E5) la diferencia entre la consulta y los documentos es que antemonemos el prefijo query: y passage: antes de computar el embedding de la consulta y de cada documento. Entonces, nuestros métodos realizaran justo eso. 

In [3]:
### Primero un ejemplo de como funciona este sentence transformer. 
### Al crear esta clase, le pasamos a la librería SentenceTransformer 
### El modelo que queremos, y que hardware usar. 

model = SentenceTransformer("intfloat/multilingual-e5-small","cpu")
model.encode("hola")

/Users/juan/downloads/ENTER/envs/st_env/lib/python3.10/site-packages/huggingface_hub/file_download.py:943: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


array([ 2.45292205e-02, -7.28081120e-03,  2.01645643e-02, -5.65277226e-02,
        8.51372033e-02, -2.35328730e-02,  4.12015505e-02,  4.14172672e-02,
        3.62707637e-02,  7.79401185e-03,  6.08525425e-02,  1.03171812e-02,
        6.10303842e-02, -6.44690078e-03, -5.33033982e-02, -2.79593244e-02,
        4.31734063e-02, -3.43652293e-02, -2.49172188e-03, -5.61207123e-02,
        2.41614040e-02, -5.16346376e-03, -4.39666212e-02,  5.65978251e-02,
        5.24599999e-02,  3.17301638e-02, -1.69265643e-02,  3.47117186e-02,
        1.48336720e-02, -8.94378573e-02, -3.34181599e-02, -3.02848890e-02,
        3.99507657e-02, -6.80900142e-02,  8.49799812e-02,  8.68255869e-02,
       -4.44168448e-02, -4.32018116e-02,  7.44791254e-02, -1.73933040e-02,
       -6.46682084e-02,  1.97995491e-02,  6.23653494e-02,  8.02007690e-02,
        5.98563179e-02,  4.08843830e-02, -6.41530827e-02, -5.04073827e-03,
       -2.49871630e-02, -3.49131376e-02, -5.10251820e-02,  6.24195635e-02,
        2.44532749e-02,  

In [4]:
### Ahora preparamos la clase para langchain, con los métodos necesarios. 

class SentenceTransformerEmbeddings(Embeddings):
    # Usamos intfloat/multilingual-e5-small, en la cpu.
    def __init__(self, model_name="intfloat/multilingual-e5-small", device="cpu", prompt="passage: "):
        self.model = SentenceTransformer(model_name, device=device)
        self.prompt = prompt

    def embed_documents(self, texts):
        # Add prompt to each text (E5 model expects it!)
        texts = [self.prompt + text for text in texts]
        return self.model.encode(texts, normalize_embeddings=True).tolist()

    def embed_query(self, text):
        # For queries, E5 expects "query: ..."
        return self.model.encode(["query: " + text], normalize_embeddings=True)[0].tolist()

In [8]:
### Y ahora creamos una instancia de esta clase. 

model_name = "intfloat/multilingual-e5-small"
device = "cpu"  # quienes quieran usar la gpu de un Apple pueden poner "mps"

model = SentenceTransformer(model_name, device=device)

### vamos a usar esto más adelante
prompts = {"query": "query: ", "passage": "passage: "}
default_prompt_name = "passage"

/Users/juan/downloads/ENTER/envs/st_env/lib/python3.10/site-packages/huggingface_hub/file_download.py:943: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


## Division en Tokens

Al momento de dividir en tokens, debemos usar el mismo modelo que para los embeddings. 

In [5]:
model_name = "intfloat/multilingual-e5-small"
tokenizer = AutoTokenizer.from_pretrained(model_name)

splitter = RecursiveCharacterTextSplitter.from_huggingface_tokenizer(
    tokenizer,
    keep_separator=True,
    add_start_index=True,
    chunk_size=512,
    chunk_overlap=0,
)

/Users/juan/downloads/ENTER/envs/st_env/lib/python3.10/site-packages/huggingface_hub/file_download.py:943: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


## Cargando un pequeño texto en Chroma. 

La base de datos computará automáticamente los vector embeddings al almacenar el texto, es una de las gracias de usar la librería langchain. 

In [6]:
### Primero dividimos el texto en párrafos

text = open("text.txt", encoding="utf-8").read()
paragraphs = text.split("\n\n") 

### Ahora cargamos el texto
documents = [Document(page_content=para) for para in paragraphs if para.strip()]

### Y ahora a dividir cada párrafo en chunks aún más pequeños, para que los maneje E5. 
### Esto no se va a ver en el texto de ejemplo, pero es importante cuando los parrafos 
### son grandes
split_chunks = splitter.split_documents(documents)

print(len(documents))
print(len(split_chunks))

10
10


In [10]:
### Ahora creamos una instancia de Chroma. No es persistente, por lo que quedará en memoria

vectorstore = Chroma.from_documents(
    documents=documents,
    embedding=SentenceTransformerEmbeddings(
        model_name="intfloat/multilingual-e5-small",
        device="cpu"
    ),
    collection_name="my_collection",
    persist_directory=None   # acá podemos hacerlo persistente
)

## Agente RAG

In [11]:
### Que LLM vamos a usar

llm_generation = ChatOpenAI(model="gpt-4.1-mini",
                 temperature=0,
                 openai_api_key=os.getenv("OPENAI_API_KEY"))

### Con que BD de vectores vamos a hacer el retrieval. 
### IMPORTANTE: search_kwargs es la cantidad de documentos que me entrega. 

retriever = vectorstore.as_retriever(search_type="similarity", search_kwargs={"k": 6})

In [12]:
### Algunas funciones que nos ayudan:

### Esta funcion llama al retriever con una user query. 
def get_context_documents(user_query, retriever):
    retrieved_docs = retriever.invoke(user_query)
    return retrieved_docs

### Formateo
def format_context_documents(docs):
    return "\n".join([doc.page_content for doc in docs])

In [13]:
### Ejemplo de uso de RAG. Con una query, el retriever nos da los docs más parecidos. 
### ¿Ves que tiene sentido?
user_query = "¿Qué animales migran largas distancias todos los años?"
docs = get_context_documents(user_query, retriever)
rag_context = format_context_documents(docs)

print(docs[0])
#print(rag_context)

page_content='Las ballenas jorobadas migran miles de kilómetros cada año entre zonas de alimentación y de reproducción.'


In [15]:
### Y aquí está la magia. Con la user_query, vamos a buscar los docs al retriever y 
### generamos un mejor prompt

def generate_agent_answer(user_query, retriever=retriever, llm_generation=llm_generation):
    docs = get_context_documents(user_query, retriever)
    rag_context = format_context_documents(docs)

    answer_prompt = f"""Given the following user question and additional context, answer the user question in Spanish.
    Question: {user_query}
    Additional Context: {rag_context}
    Answer: """

    # Respuesta LLM
    return llm_generation.invoke(answer_prompt)

In [16]:
response = generate_agent_answer("¿Qué animales migran largas distancias todos los años?", retriever=retriever, llm_generation=llm_generation)
print(response.content)

Los animales que migran largas distancias todos los años son las ballenas jorobadas y las tortugas marinas. Las ballenas jorobadas migran miles de kilómetros entre sus zonas de alimentación y reproducción, mientras que las tortugas marinas recorren grandes distancias para regresar a las playas donde nacieron y poner sus huevos.


In [18]:
### Puedes ver por que esta consulta no es respondida tan bien?
response = generate_agent_answer("De que animales o habitats tienes información", retriever=retriever, llm_generation=llm_generation)
print(response.content)

Tengo información sobre varios animales y hábitats, incluyendo las ballenas jorobadas, las tortugas marinas, el lince ibérico y las abejas. También tengo información sobre hábitats como los arrecifes de coral y la selva tropical del Amazonas.
